# Mycologue Labs - Image to 3D workflow

The following workflow demonstrates how to turn mulitple directories of images of a mushroom specimen into a high fidelity 3D models.

This workflow does not include the processing of RAW images to color corrected JPG images, which is highly recommended for quality 3D model textures.

In [ ]:
# define the locations of each image directory to process
# input_dirs = ['photos/20241023-2025-chanterelle']

import os
input_dirs = [os.path.join('photos', d) for d in os.listdir('photos') if os.path.isdir(os.path.join('photos', d))]

print(f"Directories to process: {input_dirs}")

## Step 1: Generate masks

In [ ]:
# generate masks for images
for input_dir in input_dirs:
    !python create-masks.py --input {input_dir}

## Step 2a: Generate models (serial)

In [ ]:
# create realityscan project and export 3d model
for input_dir in input_dirs:
    !python create-models.py --input {input_dir} --output models

## Step 2b: Generate models (parallel)

In [ ]:
# parrallelized model creation
import sys, subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

# realityscan allows up to 4 concurrent processes
max_workers = 4

def run_model(input_dir):
    input_dir = str(Path(input_dir).resolve())
    cmd = [sys.executable, "create-models.py", "--input", input_dir, "--output", "models"]
    cp = subprocess.run(cmd, capture_output=True, text=True)
    if cp.returncode != 0:
        raise RuntimeError(f"returncode={cp.returncode}\nSTDOUT:\n{cp.stdout}\nSTDERR:\n{cp.stderr}")
    return input_dir, cp.returncode, cp.stdout

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(run_model, d): d for d in input_dirs}
    for fut in as_completed(futures):
        d = futures[fut]
        try:
            idir, code, out = fut.result()
            print(f"✔ Finished {idir} (code {code})")
        except Exception as e:
            print(f"✘ Error on {d}:\n{e}")


## Step 3: Generate preview images and videos

In [ ]:
import os
input_models = [os.path.join('models', f) for f in os.listdir('models')
                if f.lower().endswith('100k.glb') and os.path.isfile(os.path.join('models', f))]

# input_models = ['models\\20241003-1549-russula-brevipes.100k.glb']

print(f"Previews to process: {input_models}")

Previews to process: ['models\\20241003-1549-russula-brevipes.100k.glb']


In [ ]:
for input_model in input_models:
    !blender --background --python create-preview.py -- --input {input_model} --output previews --overwrite

In [9]:
for input_model in input_models:
    !blender --background --python create-preview.py -- --video --input {input_model} --output previews --overwrite

22:23:21 | INFO: Data are loaded, start creating Blender stuff
22:23:21 | INFO: Blender create Mesh node Mesh_0_Part_0
22:23:21 | INFO: glTF import finished in 0.06s
Blender 4.5.3 LTS (hash 67807e1800cc built 2025-09-09 01:33:33)
Fra:1 Mem:147.72M (Peak 216.52M) | Time:00:01.28 | Rendering 1 / 64 samples
Fra:1 Mem:147.72M (Peak 216.52M) | Time:00:01.30 | Rendering 25 / 64 samples
Fra:1 Mem:147.72M (Peak 216.52M) | Time:00:01.32 | Rendering 50 / 64 samples
Fra:1 Mem:147.72M (Peak 216.52M) | Time:00:01.33 | Rendering 64 / 64 samples
Append frame 1
Time: 00:01.37 (Saving: 00:00.01)

Fra:2 Mem:152.29M (Peak 216.52M) | Time:00:00.04 | Rendering 1 / 64 samples
Fra:2 Mem:152.29M (Peak 216.52M) | Time:00:00.07 | Rendering 25 / 64 samples
Fra:2 Mem:152.29M (Peak 216.52M) | Time:00:00.10 | Rendering 50 / 64 samples
Fra:2 Mem:152.29M (Peak 216.52M) | Time:00:00.11 | Rendering 64 / 64 samples
Append frame 2
Time: 00:00.14 (Saving: 00:00.00)

Fra:3 Mem:152.29M (Peak 216.52M) | Time:00:00.03 | Rende